In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import IPython.display as ipd
import pyximport
pyximport.install()
%load_ext Cython

import sigkernel as ksig
from utils.midi import *
from utils.data import *
from model.generators import *

In [ ]:
hist_len = 10
sample_len = 50
seq_dim = 4
scale = 1.
stride = 10
min_notes = sample_len #NOTE: length of tensor might be longer than min_notes due to rectilinear transformation
min_gap = 0.
note_dur_transform = True

rectilinear = True
activation = 'GELU'
hidden_size = 128
n_transformer_layers = 1
n_head = 4
n_channels = 16

In [ ]:
# path = './data/maestro-v3.0.0_midi/2018/'
path = './data/midi/'
dfs = get_dfs_from_midi(path, min_notes=sample_len, min_gap=min_gap, note_dur_transform=note_dur_transform)

In [ ]:
# read pkl
dfs = pd.read_pickle('./data/dfs.pkl')

In [ ]:
# save list of dfs
import pickle
with open('./data/dfs_82.pkl', 'wb') as f:
    pickle.dump(dfs, f)

In [ ]:
dfs, pitch_range = pitch_translation(dfs)

In [ ]:
dataset = MIDIDataset(dfs, sample_len, cols=[0,1,2], scale=scale, stride=stride, rectilinear=rectilinear)
dataloader = DataLoader(dataset, batch_size=50, shuffle=True, num_workers=0, drop_last=True)

In [ ]:
generator = TransformerMusic(seq_dim, sample_len, hist_len, scale=scale, # data related
                             kernel_size=5, stride=1, n_channels=n_channels, # conv layers
                             n_head=n_head, n_transformer_layers=n_transformer_layers, # transformer layers
                             hidden_size=hidden_size, activation=activation)
generator = generator.cuda()
optimizer = torch.optim.Adam(generator.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5, verbose=True)

In [ ]:
static_kernel = ksig.static.kernels.RationalQuadraticKernel(sigma=0.1)
kernel = ksig.kernels.SignatureKernel(n_levels=5, order=5, normalization=0, static_kernel=static_kernel, device_ids=None)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for epoch in range(30):
    losses = [] # due to legacy code, losses is actually the mmd
    for batch_num, X in enumerate(tqdm(dataloader)):
        X = X.to(device)

        output = generator(X)
        X_wo_hist = X[:, hist_len:, :]

        # compute loss
        optimizer.zero_grad()
        loss = ksig.tests.mmd_loss_no_compile(X_wo_hist, output, kernel)
        losses.append(loss.item())

        # backpropagate and update weights
        loss.backward()
        optimizer.step()

    # log epoch loss and plot generated samples
    epoch_loss = np.average(losses) # average batch mmd for epoch
    scheduler.step(epoch_loss)
    print(f'Epoch {epoch}, loss: {epoch_loss}')

In [ ]:
for x in dataloader:
    x = x.to(device)
    output = generator(x)
    break

In [ ]:
i = 0
x_np = x[i].cpu().detach().numpy()
x_np[:, 2:] = np.round(x_np[:, 2:] * scale)
output_np = output[i].cpu().detach().numpy()
output_np[:, 2:] = np.round(output_np[:, 2:] * scale)
df_x = pd.DataFrame(x_np, columns=['Start', 'End', 'Pitch', 'Velocity'])
df_output = pd.DataFrame(output_np, columns=['Start', 'End', 'Pitch', 'Velocity'])

In [ ]:
df_input = df_x.iloc[hist_len:]
df_input.iloc[:,:2] = df_input.iloc[:,:2] - df_input.iloc[0,0]
df_input

In [ ]:
df_output.iloc[:,:2] = df_output.iloc[:,:2] - df_output.iloc[0,0]
df_output

In [ ]:
input_midi = df_to_midi(df_input)
output_midi = df_to_midi(df_output)

In [ ]:
Fs = 22050
audio_data = input_midi.synthesize(fs=Fs)
ipd.Audio(audio_data, rate=Fs)

In [ ]:
audio_data = output_midi.synthesize(fs=Fs)
ipd.Audio(audio_data, rate=Fs)